# Homology and Simplices
### Christina Lee
### Category: Grad
### Tags: Mathematics, Topology

I've been trying to learn the pure math side of topology to understand topological materials better.   Predictably, my sources have a lot of theorems, the examples tend to be esoteric and hard to conceptualize, and I have little idea how it all relates to my pretty crystals.  There also aren't that many examples.  I almost miss the days back in high school when they forced us to do the same problem 1,000 times over.  

So to better understand one of the important topics, simplices and simplicial complexes, I've decided to take the pure math and put it to programming the best that I can.  I don't know if this is the best way of doing things, or if it's useful at all, but I think it helps me understand them.  Hopefully, it will help you too.

<b>Homology</b> is the study of deforming things.  If we push and pull at something, without tearing or gluing, what properties remain the same?  A coffee cup is the same as a donut because they both have one hole.  

We are also the same as a donut because of our digestive track... if you ignore all the open cavities we have inside.  We can at least deform away our lungs.  I'm not an anatomy expert.  

Many objects of interest are <b>topologies</b>, which are just collections of sets that obey certain constraints.  Every manifold you've ever worked with has been a topology, but it probably was just trivial and didn't need further study.  To study more complex topologies, we can find an equivalent <b>simplicial complex</b>.  We can then compute a variety of things about the simplicial complex that tell us about the properties of the topology.  

Simplicial complexes are made out of <b>simplices</b>.  

A <b>simplex</b> is an $n$-dimensional generalization of a triangle.  A 0-dim simplex is a point; a 1-dim simplex is a line; 3-dim a tetrahedron; so on and so forth.  

![simplex](Images/simplex/simplex.svg)

The rules for the simplices in a legitimate simplicial complex are 

1. Every point must belong to at least one simplex
2. A point can belong to only a finite number of simplices
3. Two different simplices either have no points in common, or
    1. on is a face (or edge, or vertex) of the other
    2. the set of points in common is the whole of a shared face (or edge, or vertex).
            
[1]

A simplicial complex of dimension $d$ will contain a finite number of $d$-dimensional simplices, the faces of those simplices,  the faces of the faces, and so on until you get to all the points contained in the original simplices.

I will denote each point by a string of length 1, like `"a"`.

A simplex of dimension $d$ is a string of length $d$. For example, in the figure above, the two dim 2 simplices are `"abc"` and `"acd"`.  For now, order doesn't matter, but I am also working on a post to discuss <b>$p$-chains</b>, where order matters.

A simplicial complex of dimension $d$ contains simplicies of dimension $d$, dimension $d-1$, ... , to $0$. I create an array of arrays to store these simplicies. Each array of simplicies corresponds to a different dimension.

In [1]:
type simplex
    d::Int
    s::String 
end

In [2]:
type SimplexComplex
    d::Int
    s::Array{Array{simplex}}
end

After almost finishing this post and hours of staring at almost incomprehensible output, I've realized that I can use some nice IO aspects to tailor how Julia diplays my abstract types.  

I first create a function that outputs some HTML to the IO stream.  Then `Base.show` takes that IO stream and renders it as HTML.  

I believe that is what is going on.  All I know right now is this works.

I particularily used this link https://docs.julialang.org/en/stable/manual/types/#Custom-pretty-printing-1 to influence how I created this function.  

In [3]:
function displaySC(io::IO,sc::SimplexComplex)
    print(io,"<center><h3>Simplicial Complex</h3></center>")
    for kk in 1:sc.d
        print(io,"<center><b>Dimension: $(kk)</b></center>")
        sc_now=sc.s[kk]
        bigstring="<center>"
        for jj in 1:length(sc_now)
            bigstring=bigstring*"&emsp;"*sc_now[jj].s
        end
        bigstring=bigstring*"</center>"
        print(io,bigstring)
    end
end
Base.show(io::IO, ::MIME"text/html", sc::SimplexComplex)=displaySC(io,sc)

In [4]:
function displayS(io::IO,s::simplex)
    print(io,"<center><h3>Simplex</h3></center>")
    print(io,"<center><b>Dimension: $(s.d)</b></center>") 
    print(io,"<center>$(s.s)</center")
end
Base.show(io::IO,::MIME"text/html",s::simplex)=displayS(io,s)

I could just iterate out every point, edge, and face of a simplicial complex by hand, but why spend my time doing that when I can spend even more time writing code to get my computer to do it for me?

I'll send in the $d$-dimensional stucture, and the computer will figure out everything smaller than that.  

My algorithm will go through and compute the face of every block in the last array, then the edges of all the faces from before that, and so forth.  

But some edges will belong to more than one face.  See Point 3A in what a simplicial complex is, or line `"ac"` in the figure.  I could just have "ac" in the simplicial complex twice, but then things start getting ugly and clunky.  

This next function `AreSame` determines if two strings are permutations of each other, so we can clean up the complex.  The even/ odd permutation aspect will play a role next post when I discuss $p$-chains.

In [5]:
# Returns 0 is they two strings are not permutations of each other
# Returns 1 if they are even permutations
# Returns -1 if they are odd permutations

function AreSame(a::String,b::String)
    if length(a) != length(b)
        error("Strings not same length in AreSame")
    end
    
    l=length(a)
    
    x=repmat(collect(1:l),1,l)
    y=transpose(x)-1
    z=(x+y-1)%l+1
    zp=z[end:-1:1,:]

    for jj in 1:l
        n=""
        m=""
        for ii in 1:l
            n=string(n,a[z[ii,jj]]) 
            m=string(m,a[zp[ii,jj]])
        end
        
        if b==n
            return 1
        end
        if b==m
            return -1
        end
        
    end
   
    return 0
end

AreSame (generic function with 1 method)

The function to create a complex is fairly straight forward, but I want to pull out one part that is simple, but not particularily readable.  I use a loop and a modulo to remove one index from a string.  
This particularily unreadable bit is presented right here.


In [6]:
d_new_s=3
d_old_s=4
for ii in 1:d_old_s
    println(   collect(ii:(ii+d_new_s-1))%d_old_s  +1 )
end

[2,3,4]
[3,4,1]
[4,1,2]
[1,2,3]


And now the large function to create a simplicial complex.

In [7]:
# sc = simplecial complex
# s = simplex

function CreateComplex(starter::Array{simplex})
    
    n0=length(starter) # number simplices at a top dimension
    d0=starter[1].d #top dimension

    sc=Array{Array{simplex}}(d0)
    sc[d0]=starter;

    n_new_s=d0*n0 # the number of simplices in the next row
    n_old_s=n0    # the number of simplices in the last row    

    d_new_s=d0-1; # the next dimension
    d_old_s=d0;   # the last dimension

    for kk=(d0-1):-1:1
        
        s_last=sc[kk+1]
        s_next=Array{simplex}(n_new_s)

        for jj in 1:n_old_s
            for ii in 1:d_old_s

                s_next[d_old_s*(jj-1)+ii]=
                    simplex(d_new_s,s_last[jj].s[(collect(ii:(ii+d_new_s-1)))%d_old_s+1]) 

            end
        end

        sc[kk]=s_next;

        n_old_s=n_new_s
        n_new_s=n_new_s*d_new_s

        d_old_s=d_new_s;
        d_new_s=d_new_s-1;
    end
    return SimplexComplex(d0,sc)
end

CreateComplex (generic function with 1 method)

<b>Danger</b>: Because arrays are passed by reference, and how I decided to write it, `SimplifyComplex` both returns an output <b>but also MODIFIES</b> the array passed in to the function.  If you do not want your input modified, use

    SimplexComplex(sc.d,copy(sc.s))
    
as an input instead.

Now let's remove all those repeated sides.  I decided to break this into two functions for the sake of readability.

We have to be careful when deleting elements of an array we are still working with.  This means using using a `for` loop would be tricky, because my loop changes in size each time.  I had a prior implementation that had two `for` loops, but I like this way better.  To also help my situation, I delete matches from the end of the array to the beginning. 

In [8]:
function SimplifyComplex(sc::SimplexComplex)

    for kk in 1:(sc.d-1)
        sc_now=sc.s[kk]

        ii=1
        while ii<length(sc_now) 
            for jj in length(sc_now):-1:(ii+1)
                if 0 != AreSame(sc_now[jj].s,sc_now[ii].s)
                   deleteat!(sc_now,jj)
                end
            end
            ii+=1
        end
    end  

    return sc
end

SimplifyComplex (generic function with 1 method)

In [11]:
a=simplex(3,"abc")

simplex(3,"abc")

In [12]:
b=simplex(3,"abd")

simplex(3,"abd")

In [13]:
sc=CreateComplex([a,b])

SimplexComplex(3,Array{simplex,N}[simplex[simplex(1,"c"),simplex(1,"b"),simplex(1,"a"),simplex(1,"c"),simplex(1,"b"),simplex(1,"a"),simplex(1,"d"),simplex(1,"b"),simplex(1,"a"),simplex(1,"d"),simplex(1,"b"),simplex(1,"a")],simplex[simplex(2,"bc"),simplex(2,"ca"),simplex(2,"ab"),simplex(2,"bd"),simplex(2,"da"),simplex(2,"ab")],simplex[simplex(3,"abc"),simplex(3,"abd")]])

In [10]:
sc2=SimplifyComplex(sc)

SimplexComplex(3,Array{simplex,N}[simplex[simplex(1,"c"),simplex(1,"b"),simplex(1,"a"),simplex(1,"d")],simplex[simplex(2,"bc"),simplex(2,"ca"),simplex(2,"ab"),simplex(2,"bd"),simplex(2,"da")],simplex[simplex(3,"abc"),simplex(3,"abd")]])

[1] Stone, Michael, and Paul Goldbart. "Mathematics for Physics." New York: Pimander-Casaubon (2009).